In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from math import sqrt

In [2]:
def initializeWeights(n_in, n_out):
    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W

In [3]:
def sigmoid(z):
    return (1.0 / (1.0 + np.exp(-z)))

In [4]:
featureIndices=[]
def preprocess():
    mnist = loadmat('mnist_all.mat')
    train_data, train_label, validation_data, validation_label = preprocess_train_data(mnist)
    test_data, test_label = preprocess_test_data(mnist)
    
    # Feature selection
    combined  = np.concatenate((train_data, validation_data),axis=0)
    reference = combined[0,:]
    boolean_value_columns = np.all(combined == reference, axis = 0)
    
    
    featureCount = 0
    global featureIndices
    
    for i in range(len(boolean_value_columns)):
        if boolean_value_columns[i]==False:
            featureCount += 1
            featureIndices.append(i)
    print("Total number of selected features : ", featureCount)
    
    final = combined[:,~boolean_value_columns]
    tr_R = train_data.shape[0]
    vl_R = validation_data.shape[0]
    
    train_data      = final[0:tr_R,:]
    validation_data = final[tr_R:,:]
    test_data = test_data[:,~boolean_value_columns]
    
    return train_data, train_label, validation_data, validation_label, test_data, test_label

In [5]:
def preprocess_train_data(mnist):
    
    label_lst = []
    for i in range(10):
        idx = 'train'+ str(i)
        train_mat = mnist[idx]
        labels = np.full((train_mat.shape[0],1),i)
        labeled_train_mat = np.concatenate((train_mat,labels),axis=1)
        label_lst.append(labeled_train_mat)

    all_labeled_train = np.concatenate((label_lst[0],label_lst[1],label_lst[2],
                                        label_lst[3],label_lst[4],label_lst[5],
                                        label_lst[6],label_lst[7],label_lst[8],
                                        label_lst[9]), axis=0)
    
    np.random.shuffle(all_labeled_train)
    
    labeled_train = all_labeled_train[0:50000,:]
    train_data    = (labeled_train[:,0:784])/255.0
    train_label   = labeled_train[:,784]

    labeled_validation = all_labeled_train[50000:60000,:]
    validation_data    = (labeled_validation[:,0:784])/255.0
    validation_label   = labeled_validation[:,784]
    
    return train_data, train_label, validation_data, validation_label

In [6]:
def preprocess_test_data(mnist):
    label_lst = []
    for i in range(10):
        idx = 'test'+ str(i)
        test_mat = mnist[idx]
        labels = np.full((test_mat.shape[0],1),i)
        labeled_test_mat = np.concatenate((test_mat,labels),axis=1)
        label_lst.append(labeled_test_mat)

    all_labeled_test = np.concatenate((label_lst[0],label_lst[1],label_lst[2],
                                       label_lst[3],label_lst[4],label_lst[5],
                                       label_lst[6],label_lst[7],label_lst[8],
                                       label_lst[9]), axis=0)

    np.random.shuffle(all_labeled_test)
    
    test_data    = (all_labeled_test[:,0:784])/255.0
    test_label   = all_labeled_test[:,784]
    
    return test_data, test_label

In [7]:
def nnObjFunction(params, *args):
    pass

In [8]:
def nnPredict(w1, w2, data):
    pass

In [9]:
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

Total number of selected features :  717
